# Harmonizing the QSIPrep cortical/subcortical atlases

As of version 0.15.4 QSIPrep comes with a bunch of atlases that were more or less copied from their original repositories. The only strictly-enforced change to these atlases was that they were in LPS+ voxel orientation to match all other QSIPrep output files. 

There are some other annoying differences across the atlases, such as whether they are in MNI2009cAsym (the actual registration template) or FSLMNI (aka NLin6Asym) space and whether they cover the whole of gray matter (including subcortical and cerebellum). 

## Categorizing the pre 0.16 atlases

First, download the atlas directory that has been distributed with qsiprep since ~0.8:

In [1]:
%%bash

wget -nv https://upenn.box.com/shared/static/8k17yt2rfeqm3emzol5sa0j9fh3dhs0i.xz
tar xvfJm 8k17yt2rfeqm3emzol5sa0j9fh3dhs0i.xz
rm 8k17yt2rfeqm3emzol5sa0j9fh3dhs0i.xz

2022-06-01 10:25:24 URL:https://dl2.boxcloud.com/d/1/b1!SmzgLVCGSQeXTphH386fcLmSXOK5vNzgbyrS3KaeKG2p0kt7JQE1_r5cRREFfUcpPeMksFN40iadBHHCZh4Tf6xOEnKEUn9XydgZdUCisxgG3LN882hZEysJO9muLWtoewTQM7fkhPNwDWUUcS2UHwTdWC4WqDB1Ef6yE8jFrZ59V-JSjhqki6755hzJrrE8rNDcwa8KqvaqJNHSwOizCEn2UZ8UqCeNYxIXYDVx8UI4GoYSYvLxoJ5-Z-qETmoruSBQJg1ydCkSduCOq4rd-Qsdcr2CcxGhju14jp1y0oCIJRY3NdyLmpjYkfG3TvPm3uLTe2cWpQLbzj8DioagPzXNNcCWqgbuV1ysOOUsLWfwkBxV56mIPn1bdzq0O3L4-CyWYvsA2rmNVBpzcv574eiWMs-mpReUprzjcbswegFIlUyFSbpHNzPzQb0IT9v9584-nqa-sLe01hdC6oARTXU1dfqM2hASVMF3IdcITFQM0kcjN1VTUVXlQZE3zs1rVeFYZ62N9c4yt7w72eH5btMhJTZ7b2qQmL62NsRyqZfZlLatDrU3u2k4HV7Uc3lk4-sM9tBbqeLDylQA98gOS13OZpc-muINXoZs5gIt_W4ogFSAXFIbKwQFZKVio-JrHY9yKvM6OZlAuu1gZ6TBhPps3_dqze9VVQPE_p_RjK_j53DyOBAt4piYP1TVyPO4BIiB4_eYVnF8NZM-8jTDHzxP0mvCboT3KrmfFGtQkplkHbJVDneDQM3wmhBQzkSD9iAO0nv3-T7KgE-XDHtsNIb0nw0hP4kMk1JypMufHCw8FA6RbR048sZ7VwRJ47SoEE1aAGlGyFxT04rAbi2-xfUL4UsH01Iv1mBk35wFPsI69X0uLEafJjF6GYXUfUNS8_WKo1ZPWH8UMdM2jdmv3q3OobazanfA4s9fKwwWjF-pqf0N

## What are these atlases?

For reference, here is a table that describes the space of these atlases and how they appear on the brain:


| Atlas Name     | MNI Version                | Subcortical | Cerebellum | Thick   |
| :------------: | :------------------------: | :---------: | :--------: | :-----: |
|     AAL116     |           NLin6            |     Yes     |    Yes     |   Yes   |
|    Aicha384    |           NLin6            |     Yes     |     No     |   Yes   |
| Brainnetome246 |           NLin6            |     Yes     |     No     |   No    |
|   Gordon333    |           NLin6            |     No      |     No     |   NOO   |
|    power264    |           NLin6            |     Yes     |    Yes     | Spheres |
|    Schaefer    |           NLin6            |     No      |     No     |   No    |


## Harmonizing them

I cloned templateflow to get some of the transforms from NLin6Asym to 2009cAsym so that these can all be transformed to the exact MNI version that the T1ws are registered to. We also want to add subcortical + cerebellar regions to these atlases.

In [2]:
# 
import subprocess
tfbase = "/Users/mcieslak/projects/templateflow"
reference_image = "/Users/mcieslak/projects/qsiprep/qsiprep/data/mni_1mm_t1w_lps_brain.nii.gz"
transform_image = tfbase + "/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5"

def nlin6_to_2009c(nlin6_atlas, new_fname):
    subprocess.run([
        "antsApplyTransforms", "-d", "3", "-i", nlin6_atlas, "-o", new_fname,
        "-t", transform_image, "-r", reference_image, "--interpolation", "GenericLabel",
        "-v"
    ])


## Subcortical regions

We used the CTI168 atlas for subcortical regions and https://zenodo.org/record/1405484 for thalamus.


## Cerebellar regions

We downloaded the cerebellar atlas from https://github.com/DiedrichsenLab/cerebellar_atlases/blob/master/King_2019/atl-MDTB10_space-MNI_dseg.nii. This atlas is in NLin6 space and the shasum of the repository was  fd12c94.

In [3]:
needs_transform = [
    # Atlases from early qsiprep
    ('qsirecon_atlases/gordon333MNI_lps_mni.nii.gz', 
     'data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Gordon333_desc-LPS_dseg.nii.gz'),
    ('qsirecon_atlases/brainnetome246MNI_lps.nii.gz', 
     'data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Brainnetome246_desc-LPS_dseg.nii.gz'),
    ('qsirecon_atlases/aal116MNI_lps_mni.nii.gz', 
     'data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-AAL116_desc-LPS_dseg.nii.gz'),
    ('qsirecon_atlases/aicha384MNI_lps.nii.gz', 
     'data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-AICHA384_desc-LPS_dseg.nii.gz'),

    ('qsirecon_atlases/schaefer100x7MNI_lps.nii.gz',
     'data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018_desc-100Parcels7NetworksLPS_dseg.nii.gz'),
    ('qsirecon_atlases/schaefer100x17MNI_lps.nii.gz',
     'data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018_desc-100Parcels17NetworksLPS_dseg.nii.gz'),

    ('qsirecon_atlases/schaefer200x7MNI_lps.nii.gz',
     'data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018_desc-200Parcels7NetworksLPS_dseg.nii.gz'),
    ('qsirecon_atlases/schaefer200x17MNI_lps.nii.gz',
     'data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018_desc-200Parcels17NetworksLPS_dseg.nii.gz'),

    ('qsirecon_atlases/schaefer400x7MNI_lps.nii.gz',
     'data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018_desc-400Parcels7NetworksLPS_dseg.nii.gz'),
    ('qsirecon_atlases/schaefer400x17MNI_lps.nii.gz',
     'data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018_desc-400Parcels17NetworksLPS_dseg.nii.gz'),

    # New image
    ('data/NLin6Asym/atl-MDTB10_space-MNI_dseg.nii', 
     'data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-MDTB10_desc-LPS_dseg.nii.gz'),
    
    # Thalamus parcellation
    ('data/NLin6Asym/Thalamus_Nuclei-HCP-MaxProb.nii.gz',
     'data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-ThalamusHCP_desc-LPS_dseg.nii.gz')
]

for source_img, dest_img in needs_transform:
    nlin6_to_2009c(source_img, dest_img)

Using double precision for computations.
Input scalar image: qsirecon_atlases/gordon333MNI_lps_mni.nii.gz
Reference image: /Users/mcieslak/projects/qsiprep/qsiprep/data/mni_1mm_t1w_lps_brain.nii.gz
The composite transform comprises the following transforms (in order): 
  1. /Users/mcieslak/projects/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5[0] (type = AffineTransform)
  2. /Users/mcieslak/projects/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5[1] (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: LabelImageGenericInterpolateImageFunction
Output warped image: data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Gordon333_desc-LPS_dseg.nii.gz
Using double precision for computations.
Input scalar image: qsirecon_atlases/brainnetome246MNI_lps.nii.gz
Reference image: /Users/mcieslak/projects/qsiprep/qsiprep/data/mni_1mm_t1w_lps_brain.nii.gz
The composite t

Output warped image: data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018_desc-400Parcels7NetworksLPS_dseg.nii.gz
Using double precision for computations.
Input scalar image: qsirecon_atlases/schaefer400x17MNI_lps.nii.gz
Reference image: /Users/mcieslak/projects/qsiprep/qsiprep/data/mni_1mm_t1w_lps_brain.nii.gz
The composite transform comprises the following transforms (in order): 
  1. /Users/mcieslak/projects/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5[0] (type = AffineTransform)
  2. /Users/mcieslak/projects/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5[1] (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: LabelImageGenericInterpolateImageFunction
Output warped image: data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018_desc-400Parcels17NetworksLPS_dseg.nii.gz
Using double precision for computations.
Input scalar image: dat

We just need to change the orientation on the subcortical atlases. 

In [4]:
%%bash
3dresample -orient RAI \
    -overwrite \
    -inset data/2009cAsym/CIT168toMNI152-2009c_det.nii.gz \
    -prefix data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-CIT168_desc-LPS_dseg.nii.gz

## Check that they're all in exactly the same voxel grid

We need to be certain that the voxel grids are exactly the same so we can start swapping in the cerebellum and subcortical regions

In [5]:
from glob import glob
import nibabel as nb
import numpy as np

print("checking files:")
atlas_files = [fname for fname in sorted(glob("data/2009cAsym/*LPS*gz")) 
               if ("MDT" not in fname) and ("CoBra" not in fname)]
print("\n".join(atlas_files))

print("\ncomparing grids:")
ref_img = nb.load(atlas_files.pop())

for try_file in atlas_files:
    try_img = nb.load(try_file)
    if not np.allclose(try_img.affine, ref_img.affine):
        raise Exception('incompatible affines:', try_img, try_img.affine, ref_img.affine)
    if not try_img.shape == ref_img.shape:
        raise Exception('incompatible shapes:', try_img, try_img.shape, ref_img.shape)
print("All grids match orientation and shape!!")

checking files:
data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-AAL116_desc-LPS_dseg.nii.gz
data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-AICHA384_desc-LPS_dseg.nii.gz
data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Brainnetome246_desc-LPS_dseg.nii.gz
data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-CIT168_desc-LPS_dseg.nii.gz
data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-CoBrALabSubCortical_desc-LPS_dseg.nii.gz
data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Gordon333_desc-LPS_dseg.nii.gz
data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018_desc-100Parcels17NetworksLPS_dseg.nii.gz
data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018_desc-100Parcels7NetworksLPS_dseg.nii.gz
data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018_desc-200Parcels17NetworksLPS_dseg.nii.gz
data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018_desc-200Parcels7NetworksLPS_dseg.nii.gz
data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Scha

## How are atlases currently represented

The tarball containing the atlas images also contains `atlas_config.json`

In [6]:
import json
original_atlas_config_f = "qsirecon_atlases/atlas_config.json"
with open(original_atlas_config_f, "r") as configf:
    original_atlas_config = json.load(configf)

print("\n".join(original_atlas_config.keys()))

gordon333
power264
schaefer100
schaefer200
schaefer400
aal116
aicha384
brainnetome246
schaefer100x7
schaefer100x17
schaefer200x7
schaefer200x17
schaefer400x7
schaefer400x17


Let's look at what one of these entries looks like:

In [7]:
aal_config = original_atlas_config['aal116']
print(aal_config.keys())
print(aal_config['node_names'][:10])
print(aal_config['node_ids'][:10])
print(aal_config['file'])

dict_keys(['file', 'node_names', 'node_ids'])
['Precentral_L', 'Precentral_R', 'Frontal_Sup_L', 'Frontal_Sup_R', 'Frontal_Sup_Orb_L', 'Frontal_Sup_Orb_R', 'Frontal_Mid_L', 'Frontal_Mid_R', 'Frontal_Mid_Orb_L', 'Frontal_Mid_Orb_R']
[2001, 2002, 2101, 2102, 2111, 2112, 2201, 2202, 2211, 2212]
aal116MNI_lps_mni.nii.gz


### The `'atlas'` key

Each atlas has a key. In this case we looked at the `'aal116'` key. This is what you would put in the json file for your reconstruction workflow 's `"atlases"` field. For example you could see something like

```json
{
  "name": "my_custom_connectome_workflow",
  "atlases": ["aal116"],
  ...
}
```

### The `'node_ids'` and `'node_names'` entries

These two lists are paired by index, such that `aal_config['node_ids'][i]` is the integer value of the ROI in the nifti file `aal_config['file']` and its description can be found in `aal_config['node_names'][i]`. In the case of the Schaefer atlas there are two possible names per node, so we'll add a second `'node_names'`.

Here is a function that verifies that an atlas is completely specified:

In [8]:
import numpy as np

def verify_atlas(atlas_config, atlas_data):
    data_regions = np.unique(atlas_data)
    data_regions = data_regions[data_regions > 0].tolist()
    for node_id, node_name in zip(atlas_config['node_ids'], atlas_config['node_names']):
        if node_id not in data_regions:
            raise Exception("%d: %s not in atlas data" % (node_id, node_name))
    
    if not len(atlas_config['node_ids']) == len(atlas_config['node_names']):
        raise Exception("Inconsistent number of node names and ids")
    
    missing_regions = set(atlas_config['node_ids']).difference(data_regions)
    if  missing_regions:
        raise Exception("%s present in data but not in labels" % str(missing_regions))

## Preparing the subcortical atlases

### CIT168

There are a couple problems with this atlas for our purposes
  
  1. The ROIs are not separated into Left and Right hemisphere
  2. Regions 7, 10 and 11 are not likely to survive resampling
  3. There is no thalamus or hippocampus
  4. Region 15 is likely to be outside of any brain mask

We need to get these into shape so we can add them to the other atlases. In particular the subcortical atlas is a little weird. We need to split it into left and right hemispheres.

In [9]:
# Do CIT168
cit168_subcortical = {
    "node_names": [],
    "node_ids": []
}

with open("data/2009cAsym/CIT168labels.txt") as f:
    for line in f:
        node_id, node_name = line.strip().split(",")
        cit168_subcortical['node_names'].append("CIT168Subcortical_" + node_name)
        cit168_subcortical['node_ids'].append(int(node_id)+1) # they started at 0, wtf

cit168_data = nb.load(
    "data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-CIT168_desc-LPS_dseg.nii.gz"
).get_fdata().astype(np.uint32)


# Merge the smaller regions together
cit_lut = {node_id: region_label for node_id, region_label in 
           zip(cit168_subcortical['node_ids'], 
               cit168_subcortical['node_names'])}
cit168_data[cit168_data==10] = 7
cit168_data[cit168_data==11] = 7
cit_region7_label = "_".join([cit_lut[7], cit_lut[10], cit_lut[11]])
cit_lut[7] = cit_region7_label
del cit_lut[10], cit_lut[11]
merged_cit_ids = sorted(cit_lut.keys())
merged_cit_labels = [cit_lut[node_id] for node_id in merged_cit_ids]
merged_cit_config = {"node_ids": merged_cit_ids, "node_names":merged_cit_labels}
verify_atlas(merged_cit_config, cit168_data)

# Add 100 to the LH regions to separate them
offsetmask = np.zeros_like(cit168_data)
midvoxel = offsetmask.shape[0] // 2
offsetmask[midvoxel:, :, :] = 100
hemi_cit_data = (cit168_data + offsetmask) * (cit168_data > 0)

# Split regions into hemispheres
cit_hemi_labels = [name + "_rh" for name in merged_cit_labels] + \
             [name + "_lh" for name in merged_cit_labels]
cit_hemi_ids = merged_cit_ids + [_id + 100 for _id in merged_cit_ids]
cit_hemi_config = {
    "node_ids": cit_hemi_ids,
    "node_names": cit_hemi_labels
}
verify_atlas(cit_hemi_config, hemi_cit_data)


### Thalamic segmentation

This segmentation redoes the thalamus tractography parcellation with better data. It is also much better masked on the thalamus than other atlases
  
  


In [10]:
# Do Thalamus

thalamus_atlas = {
    "node_names": [],
    "node_ids": []
}

with open("data/2009cAsym/Thalamic_Nuclei-labels.txt") as f:
    for line in f:
        node_id, node_name = line.strip().split(",")
        thalamus_atlas['node_names'].append("Thalamus_" + node_name)
        thalamus_atlas['node_ids'].append(int(node_id))

thalamus_data = nb.load(
    "data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-ThalamusHCP_desc-LPS_dseg.nii.gz"
).get_fdata().astype(np.uint32)

# Check it
verify_atlas(thalamus_atlas, thalamus_data)

### The king cerebellum atlas

This one has just cerebellum labels

In [11]:
# Now the cerebellum atlas
king_cerebellum = {
    "node_names": [],
    "node_ids": [] 
}

with open("data/2009cAsym/cerebellum_regions.tsv") as f:
    for line in f:
        node_id, node_name, color = line.strip().split()
        king_cerebellum['node_names'].append("Cerebellum_" + node_name)
        king_cerebellum['node_ids'].append(int(node_id))
king_cb_data = nb.load(
    "data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-MDTB10_desc-LPS_dseg.nii.gz"
).get_fdata().astype(np.uint32)

# Check this too
verify_atlas(king_cerebellum, king_cb_data)

## Combining the atlases

We need to add the cerebellum, thalamus and subcortical regions into a single atlas. Below are some functions that help us add one atlas into another. We'll add these three together and then add them to all the other atlases that don't have any subcortical regions

In [12]:
from copy import deepcopy
from scipy.stats import mode

def match_partitions(data1, data2):
    """Match the regions in data1 to the regions in data2"""
    
    data1_ids = np.unique(data1)
    data1_ids = data1_ids[data1_ids > 0].tolist()
    data2_ids = np.unique(data2)
    data2_ids = data2_ids[data2_ids > 0].tolist()
    
    mapping = {}
    for data1_id in data1_ids:
        region_mask = data1 == data1_id
        mapping[data1_id], = mode(data2[region_mask]).mode
    
    
    return mapping

def remap_values(original_data, mapping):
    remapped = np.zeros_like(original_data)
    for old_value, new_value in mapping.items():
        remapped[original_data==old_value] = new_value
    return remapped


Testing this code, let's use the cit168 atlas as the base and make sure these functions do what we think they do

In [13]:
# This simulates re-numbering the 
new_atlas_config = cit_hemi_config
new_atlas_mapping = {old_value: new_value for old_value, new_value in 
                    zip(new_atlas_config['node_ids'],
                        np.argsort(new_atlas_config['node_ids'])+1)}

print("atlas mapping:")
print(*sorted(new_atlas_mapping.items()), sep='  ')

remapped_atlas = remap_values(hemi_cit_data, new_atlas_mapping)
print("matched labels:" )
print(*sorted(match_partitions(hemi_cit_data, remapped_atlas).items()), sep='  ')

atlas mapping:
(1, 1)  (2, 2)  (3, 3)  (4, 4)  (5, 5)  (6, 6)  (7, 7)  (8, 8)  (9, 9)  (12, 10)  (13, 11)  (14, 12)  (15, 13)  (16, 14)  (101, 15)  (102, 16)  (103, 17)  (104, 18)  (105, 19)  (106, 20)  (107, 21)  (108, 22)  (109, 23)  (112, 24)  (113, 25)  (114, 26)  (115, 27)  (116, 28)
matched labels:
(1, 1)  (2, 2)  (3, 3)  (4, 4)  (5, 5)  (6, 6)  (7, 7)  (8, 8)  (9, 9)  (12, 10)  (13, 11)  (14, 12)  (15, 13)  (16, 14)  (101, 15)  (102, 16)  (103, 17)  (104, 18)  (105, 19)  (106, 20)  (107, 21)  (108, 22)  (109, 23)  (112, 24)  (113, 25)  (114, 26)  (115, 27)  (116, 28)


It works!! Now we can start adding these atlases to one another.



In [14]:
def add_atlas_to_another(atlas1_config, atlas1_data, 
                         atlas2_config, atlas2_data):
    """
    Add atlas2 into atlas1. Ensure the labels are updated
    so there is no overlap 
    """

    # Verify inputs
    verify_atlas(atlas1_config, atlas1_data)
    verify_atlas(atlas2_config, atlas2_data)
    atlas2_data = atlas2_data.copy()

    # What is the largest number in atlas1? This will be the minimum
    # value in the new ids for atlas2 after it's been added to atlas1
    merged_atlas_min = np.max(atlas1_data) + 2
    atlas2_id_mapping = dict(
        zip(atlas2_config['node_ids'],
            np.argsort(atlas2_config['node_ids']) + merged_atlas_min))
    atlas2_merged_ids = [atlas2_id_mapping[node_id] for node_id in
                         atlas2_config['node_ids']]

    # Make sure that any voxels that are labeled in the base image 
    # are NOT overwritten by the new atlas data
    atlas2_data[atlas1_data > 0] = 0

    # Ensure that we have not clobbered any regions by doing so
    verify_atlas(atlas2_config, atlas2_data)

    # Change their node ids so they don't conflict with the base
    remapped_atlas2 = remap_values(atlas2_data, atlas2_id_mapping)

    merged_image_data = atlas1_data + remapped_atlas2
    merged_image_labels = atlas1_config['node_names'] + \
                          atlas2_config['node_names']
    merged_image_ids = atlas1_config['node_ids'] + atlas2_merged_ids
    merged_atlas_config = {"node_names": merged_image_labels,
                           "node_ids": merged_image_ids}
    verify_atlas(merged_atlas_config, merged_image_data)
    return merged_atlas_config, merged_image_data



### Creating the full subcortical atlas

Here we merge the three subcortical atlases

In [15]:
thalamus_and_cit_config, thalamus_and_cit_data = add_atlas_to_another(
    cit_hemi_config, hemi_cit_data,
    thalamus_atlas, thalamus_data,
)

full_subcortical_config, full_subcortical_data = add_atlas_to_another(
    thalamus_and_cit_config, thalamus_and_cit_data,
    king_cerebellum, king_cb_data
)

subcortical_img = nb.Nifti1Image(full_subcortical_data, 
                                 ref_img.affine, 
                                 header=ref_img.header)
subcortical_img.to_filename("subcortical_regions.nii.gz")

## Add the subcortical atlas to cortical parcellations

Now we can add these to the cortical atlases that are missing subcortical regions.

In [16]:
import os
os.makedirs("verified_atlases", exist_ok=True)
def save_data(data_matrix, fname):
    img = nb.Nifti1Image(data_matrix, 
                         ref_img.affine, 
                         header=ref_img.header)
    img.to_filename("verified_atlases/" + fname)

# The new atlas_config.json
final_config = {}

# schaefer100
schaefer100x7_config = original_atlas_config['schaefer100x7']
schaefer100x7_file = 'data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018_desc-100Parcels7NetworksLPS_dseg.nii.gz'
schaefer100x7_data = nb.load(schaefer100x7_file).get_fdata().astype(np.uint32)
verify_atlas(schaefer100x7_config, schaefer100x7_data)
schaefer100x7_sc_config, schaefer100x7_sc_data = add_atlas_to_another(
    schaefer100x7_config, schaefer100x7_data,
    full_subcortical_config, full_subcortical_data)
schaefer100x7_sc_file = 'tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018Ext_desc-100Parcels7NetworksLPS_dseg.nii.gz'
schaefer100x7_sc_config['file'] = schaefer100x7_sc_file
save_data(schaefer100x7_sc_data, schaefer100x7_sc_file)
final_config['schaefer100'] = schaefer100x7_sc_config

# Schaefer200
schaefer200x7_config = original_atlas_config['schaefer200x7']
schaefer200x7_file = 'data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018_desc-200Parcels7NetworksLPS_dseg.nii.gz'
schaefer200x7_data = nb.load(schaefer200x7_file).get_fdata().astype(np.uint32)
verify_atlas(schaefer200x7_config, schaefer200x7_data)
schaefer200x7_sc_config, schaefer200x7_sc_data = add_atlas_to_another(
    schaefer200x7_config, schaefer200x7_data,
    full_subcortical_config, full_subcortical_data)
schaefer200x7_sc_file = 'tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018Ext_desc-200Parcels7NetworksLPS_dseg.nii.gz'
schaefer200x7_sc_config['file'] = schaefer200x7_sc_file
save_data(schaefer200x7_sc_data, schaefer200x7_sc_file)
final_config['schaefer200'] = schaefer200x7_sc_config

# Schaefer400
schaefer400x7_config = original_atlas_config['schaefer400x7']
schaefer400x7_file = 'data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018_desc-400Parcels7NetworksLPS_dseg.nii.gz'
schaefer400x7_data = nb.load(schaefer400x7_file).get_fdata().astype(np.uint32)
verify_atlas(schaefer400x7_config, schaefer400x7_data)
schaefer400x7_sc_config, schaefer400x7_sc_data = add_atlas_to_another(
    schaefer400x7_config, schaefer400x7_data,
    full_subcortical_config, full_subcortical_data)
schaefer400x7_sc_file = 'tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018Ext_desc-400Parcels7NetworksLPS_dseg.nii.gz'
schaefer400x7_sc_config['file'] = schaefer400x7_sc_file
save_data(schaefer400x7_sc_data, schaefer400x7_sc_file)
final_config['schaefer400'] = schaefer400x7_sc_config

In [17]:
# Gordon333
Gordon333_config = original_atlas_config['gordon333']
Gordon333_file = 'data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Gordon333_desc-LPS_dseg.nii.gz'
Gordon333_data = nb.load(Gordon333_file).get_fdata().astype(np.uint32)
verify_atlas(Gordon333_config, Gordon333_data)
Gordon333_sc_config, Gordon333_sc_data = add_atlas_to_another(
    Gordon333_config, Gordon333_data,
    full_subcortical_config, full_subcortical_data)
Gordon333_sc_file = 'tpl-MNI152NLin2009cAsym_res-01_atlas-Gordon333Ext_desc-LPS_dseg.nii.gz'
Gordon333_sc_config['file'] = Gordon333_sc_file
save_data(Gordon333_sc_data, Gordon333_sc_file)
final_config['gordon333'] = Gordon333_sc_config

In [18]:
# Add cb to aicha
aicha384_config = original_atlas_config['aicha384']
aicha384_file = 'data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-AICHA384_desc-LPS_dseg.nii.gz'
aicha384_data = nb.load(aicha384_file).get_fdata().astype(np.uint32)
verify_atlas(aicha384_config, aicha384_data)

aicha384_sc_config, aicha384_sc_data = add_atlas_to_another(
    aicha384_config, aicha384_data,
    king_cerebellum, king_cb_data)

aicha384_sc_file = 'tpl-MNI152NLin2009cAsym_res-01_atlas-AICHA384Ext_desc-LPS_dseg.nii.gz'
aicha384_sc_config['file'] = aicha384_sc_file
save_data(aicha384_sc_data, aicha384_sc_file)
final_config['aicha384'] = aicha384_sc_config

In [19]:
# Add cb to brainnetome
brainnetome246_config = original_atlas_config['brainnetome246']
brainnetome246_file = 'data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-Brainnetome246_desc-LPS_dseg.nii.gz'
brainnetome246_data = nb.load(brainnetome246_file).get_fdata().astype(np.uint32)
brainnetome246_sc_config, brainnetome246_sc_data = add_atlas_to_another(
    brainnetome246_config, brainnetome246_data,
    king_cerebellum, king_cb_data)

brainnetome246_sc_file = 'tpl-MNI152NLin2009cAsym_res-01_atlas-Brainnetome246Ext_desc-LPS_dseg.nii.gz'
brainnetome246_sc_config['file'] = brainnetome246_sc_file
save_data(brainnetome246_sc_data, brainnetome246_sc_file)
final_config['brainnetome246'] = brainnetome246_sc_config

In [20]:
# AAL is it's own whole thing, just copy it over

aal116_config = original_atlas_config['aal116']
aal116_file = 'data/2009cAsym/tpl-MNI152NLin2009cAsym_res-01_atlas-AAL116_desc-LPS_dseg.nii.gz'
aal116_data = nb.load(aal116_file).get_fdata().astype(np.uint32)
verify_atlas(aal116_config, aal116_data)

aal116_sc_file = 'tpl-MNI152NLin2009cAsym_res-01_atlas-AAL116_desc-LPS_dseg.nii.gz'
aal116_config['file'] = aal116_sc_file
save_data(aal116_data, aal116_sc_file)
final_config['aal116'] = aal116_config


os.chdir("verified_atlases")

In [21]:

for config in final_config:
    data = nb.load(final_config[config]['file']).get_fdata().astype(np.uint32)
    verify_atlas(final_config[config], data)
    print("Verified", config)

Verified schaefer100
Verified schaefer200
Verified schaefer400
Verified gordon333
Verified aicha384
Verified brainnetome246
Verified aal116


In [22]:
for key in final_config:
    final_config[key]['node_ids'] = list(map(int, final_config[key]['node_ids']))

with open('atlas_config.json', 'w') as f:
    json.dump(final_config, f, indent=4)

In [23]:
%%bash

cd ..
mkdir tmp
mv verified_atlases tmp/qsirecon_atlases
cd tmp
tar cvfJ qsiprep-atlases1.3.tar.xz qsirecon_atlases
mv qsiprep-atlases1.3.tar.xz ../
cd ..
rm -rf tmp

a qsirecon_atlases
a qsirecon_atlases/tpl-MNI152NLin2009cAsym_res-01_atlas-AAL116_desc-LPS_dseg.nii.gz
a qsirecon_atlases/tpl-MNI152NLin2009cAsym_res-01_atlas-Gordon333Ext_desc-LPS_dseg.nii.gz
a qsirecon_atlases/tpl-MNI152NLin2009cAsym_res-01_atlas-AICHA384Ext_desc-LPS_dseg.nii.gz
a qsirecon_atlases/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018Ext_desc-400Parcels7NetworksLPS_dseg.nii.gz
a qsirecon_atlases/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018Ext_desc-200Parcels7NetworksLPS_dseg.nii.gz
a qsirecon_atlases/atlas_config.json
a qsirecon_atlases/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018Ext_desc-100Parcels7NetworksLPS_dseg.nii.gz
a qsirecon_atlases/tpl-MNI152NLin2009cAsym_res-01_atlas-Brainnetome246Ext_desc-LPS_dseg.nii.gz
